# R3T1

# Openning Data

In [ ]:
using SparseArrays
using JuMP
using Ipopt
using GLPK
using LinearAlgebra
using MAT

In [ ]:
file = matopen("R3T1.mat")
varnames = names(file)

In [ ]:
S = read(file, "S")
L = read(file, "L")
U = read(file, "U");

In [ ]:
(m,n)=size(S)
(r,c) = size(L)

# constructing model

In [ ]:
mzeros = zeros(m,1);

In [ ]:
# main function
function opt_vector1(S,l,u)
    model = Model(Ipopt.Optimizer)
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    @objective(model,Min,sum(a[i] for i=1:r))

    optimize!(model)
    
    return JuMP.value.(v)
end

In [ ]:
# this function took too long that we terminated the run.
function opt_vector2(S,L,U)
    model = Model(GLPK.Optimizer)
    @variable(model,V[i=1:r,j=1:c])
    
    @constraint(model, S*V .== mzeros)
    @constraint(model, U .>= V)
    @constraint(model, V .>= L)
    @objective(model,Min,sum(V))

    optimize!(model)
    
    return JuMP.value.(V)
end

In [ ]:
# we came up this idea after the end of this round
function opt_vector3(S,l,u,z)
    model = Model(Ipopt.Optimizer)
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    @NLobjective(model,Min,sum(log(1+z[i]+a[i]) for i=1:r))
    optimize!(model)
    
    return JuMP.value.(v)
end

In [ ]:
function opt_vector4(S,l,u,z)
    model = Model(Ipopt.Optimizer)
    @variable(model, l[i] <= v[i = 1:r] <= u[i])
    @variable(model,a[1:r])
    @constraint(model, S*v .== mzeros)
    @constraint(model, a .>= v)
    @constraint(model, a .>= -v)
    @NLobjective(model,Min,sum(z[i]*a[i] for i=1:r))
    optimize!(model)
    
    return JuMP.value.(v)
end

In [ ]:
function update_weights(w,v,delta)
    n = length(w)
    phi = zeros(n,1)
    for i=1:n
        if(abs(delta+v[i])>10^(-4))
            phi[i] = delta/(delta+v[i])
        else
            phi[i]=w[i]
        end
    end
    return phi
end

In [ ]:
function zero_rows(V)
    indexes= zeros(Int64,size(V)[1],1)
    k=1
    for i in 1:size(V)[1]
        if sum(V[i,:].*V[i,:])<1e-10
            indexes[k]= i
            k+=1
        end
    end
    return length(indexes[1:k-1])
end

In [ ]:
#this function computes l2-norm of each row and returns a vector of norms
function row_squares(V)
    r,c = size(V,1), size(V,2)
    squares = zeros(r,1)
    for i in 1:r
        z=0
        for j in 1:c
           z += (V[i,j])^2
        end
        squares[i]=sqrt(z)
    end
    return squares
end

## running code with the function that we came up after the deadline of this round

In [ ]:
l = L[:,1]
u = U[:,1]
z = ones(r,1)
v = opt_vector3(S,l,u,z)
#v = opt_vector1(S,l,u)
zero_rows(v)

In [ ]:
V=v;

In [ ]:
for i in 2:c
    if isdefined(Main, :IJulia)
        Main.IJulia.stdio_bytes[] = 0
    end
    l = L[:,i]
    u = U[:,i]
    z = row_squares(V)
    v = opt_vector3(S,l,u,z)
    #v = opt_vector1(S,l,u)
    V = [V v]
    print("**i : ",i,"\n")
    print("***",zero_rows(V),"\n") 
end

In [ ]:
zero_rows(V)

#### number of zero rows are more than the answer we submitted

In [ ]:
V

# writing to output file

In [ ]:
using DelimitedFiles
open("output.txt","w") do io
    writedlm(io, V,",")
end